### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

### Functions

In [ ]:
# variable plotting

def attribute_plot(data, time, attribute, plot_name):
    plt.figure(figsize=(10,5))
    plt.plot(data[time], data[attribute])
    #labels = data[time]
    #plt.legend()
    plt.xlabel(xlabel=time)
    plt.ylabel(ylabel=attribute)
    #plt.xticks(data[time], labels, rotation='vertical')
    plt.title(label=plot_name)
    plt.grid(True)
    
# distribution plotting

def distribution(data, attribute, title):
    plt.figure(figsize=(10,5))
    sns.histplot(data[attribute], alpha=0.5)
    plt.axvline(data[attribute].median(), color='r', linestyle='dashed', linewidth=2, label='median value')
    plt.axvline(data[attribute].mean(), color='purple', linestyle='dashed', linewidth=2, label='average value')
    plt.axvline(data[attribute].quantile(0.25), color='r', linestyle='dotted', linewidth=3, label='25% and 75% values')
    plt.axvline(data[attribute].quantile(0.75), color='r', linestyle='dotted', linewidth=3)
    plt.legend()
    plt.title(label=title)
    plt.show()

### Import dependencies

In [ ]:
data = pd.read_csv('train_data.csv')
data

In [ ]:
# what features we have?

for i in data.columns:
    print(i)

In [ ]:
data = data.drop(columns='index')

### Data types verification

In [ ]:
data.info()

In [ ]:
data.dtypes

In [ ]:
# change date format from 'object' to 'datetime'

data['startdate'] = pd.to_datetime(data['startdate'])
data.dtypes

In [ ]:
# what feature has type 'object' besides the date?

data.select_dtypes(include=['object'])

### Missing values and duplicates check

In [ ]:
data.isna().sum().any()

In [ ]:
data.duplicated().any()

In [ ]:
missing_values = data.isna().sum()
missing_values = missing_values[missing_values > 0]
missing_values

There are 8 variables with missing values. All of them related to weather models forecasting.

### Features description

Date: startdate

Static features
Geoposition: lat, lon, elevation__elevation, climateregions__climateregion

Dynamics features

1. Environmental features (humidity, pressure, water): contest-pevpr-sfc-gauss-14d__pevpr – evaporation, contest-rhum-sig995-14d__rhum – relative humidity, contest-slp-14d__slp – sea level pressure, contest-pres-sfc-gauss-14d__pres – pressure, contest-prwtr-eatm-14d__prwtr – precipitable water for entire atmosphere, contest-precip-14d__precip – measured precipitation

2. Wind:
- Longitudinal wind: contest-wind-vwnd-925-14d__wind-vwnd-925, contest-wind-vwnd-250-14d__wind-vwnd-250
- Zonal wind: contest-wind-uwnd-250-14d__wind-uwnd-250, contest-wind-uwnd-925-14d__wind-uwnd-925
- Geopotential height wind: contest-wind-h10-14d__wind-hgt-10 – height 10,contest-wind-h100-14d__wind-hgt-100  – height 100, contest-wind-h500-14d__wind-hgt-500 – height 500, contest-wind-h850-14d__wind-hgt-850 – height 850

3. Temperature - target: the arithmetic mean of the max and min observed temperature over the next 14 days for each location and start date: contest-tmp2m-14d__tmp2m 

4. Temperature computed by weather models:
- recent forecasts from different weather models: cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean
- sea surface temperature: sst-2010-1 - sst-2010-10
- most recent monthly NMME model forecasts for target and average forecast across those models (nmme0mean): nmme0-tmp2m-34w__cancm30, nmme0-tmp2m-34w__cancm40, nmme0-tmp2m-34w__ccsm30, nmme0-tmp2m-34w__ccsm40, nmme0-tmp2m-34w__cfsv20, nmme0-tmp2m-34w__gfdlflora0, nmme0-tmp2m-34w__gfdlflorb0, nmme0-tmp2m-34w__gfdl0, nmme0-tmp2m-34w__nasa0, nmme0-tmp2m-34w__nmme0mean
- weeks 3-4 weighted average of most recent monthly NMME model forecasts for target label: nmme-tmp2m-34w__cancm3, nmme-tmp2m-34w__cancm4, nmme-tmp2m-34w__ccsm3, nmme-tmp2m-34w__ccsm4, nmme-tmp2m-34w__cfsv2, nmme-tmp2m-34w__gfdl, nmme-tmp2m-34w__gfdlflora, nmme-tmp2m-34w__gfdlflorb, nmme-tmp2m-34w__nasa, nmme-tmp2m-34w__nmmemean
- weeks 5-6 weighted average of most recent monthly NMME model forecasts for target label: nmme-tmp2m-56w__cancm3, nmme-tmp2m-56w__cancm4, nmme-tmp2m-56w__ccsm3, nmme-tmp2m-56w__ccsm4, nmme-tmp2m-56w__cfsv2, nmme-tmp2m-56w__gfdl, nmme-tmp2m-56w__gfdlflora, nmme-tmp2m-56w__gfdlflorb, nmme-tmp2m-56w__nasa, nmme-tmp2m-56w__nmmemean

5. Precipitation computed by weather models:
- weeks 3-4 weighted average of monthly NMME model forecasts for precipitation: nmme-prate-34w from weather models cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean
- weeks 5-6 weighted average of monthly NMME model forecasts for precipitation: nmme-prate-56w from weather models cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean
- weeks 3-4 weighted average of most recent monthly NMME model forecasts for precipitation: nmme0-prate-34w from weather models cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean
- weeks 5-6 weighted average of most recent monthly NMME model forecasts for precipitation: nmme0-prate-56w from weather models cancm30, cancm40, ccsm30, ccsm40, cfsv20, gfdlflora0, gfdlflorb0, gfdl0, nasa0, nmme0mean

Features related to El Nino:
1. Sea ice concentration: icec-2010-1 - icec-2010-10, where 1-10 - years after the El Nino
2. Oscillation coefficients:
- Madden-Julian oscillation: mjo1d__phase, mjo1d__amplitude
- Multivariate ENSO index (MEI) (associated with El Nino/Southern Oscillation (ENSO)): mei__mei, mei__meirank, mei__nip
3. Wind
- Longitudinal wind: wind-vwnd-250-2010-1 - wind-vwnd-250-2010-20, wind-vwnd-925-2010-1 - wind-vwnd-925-2010-20
- Zonal wind: wind-uwnd-250-2010-1 - wind-uwnd-250-2010-10
- Geopotential height wind: wind-hgt-10-2010-1 - wind-hgt-10-2010-10, wind-hgt-100-2010-1 - wind-hgt-100-2010-10, wind-hgt-500-2010-1 - wind-hgt-500-2010-10, wind-hgt-850-2010-1 - wind-hgt-850-2010-10

### Target investigation

In [ ]:
# five point summary

data['contest-tmp2m-14d__tmp2m'].describe()

In [ ]:
attribute_plot(data, 'startdate', 'contest-tmp2m-14d__tmp2m', 'Target')

In [ ]:
distribution(data, 'contest-tmp2m-14d__tmp2m', 'Target distribution')

The target (temperature) has a distribution similar to the normal. However, the average value is not equal to the median value.

### Locations investigation

In [ ]:
# How many locations we have?

data.groupby(['lat', 'lon'], as_index=False, sort=False).aggregate('count')

We have 514 locations and 731 measurements for each of them.

In [ ]:
plt.scatter(data['lon'], data['lat'])
plt.xlabel(xlabel='latitude')
plt.ylabel(ylabel='longitude')
plt.title(label='Locations')

### Climatic regions investigation

In [ ]:
# How many climatic regions we have?

data.groupby('climateregions__climateregion', as_index=False).aggregate('count')

15 climatic regions and different measurements for each of them.

In [ ]:
data['climateregions__climateregion'].value_counts().plot(kind='bar', rot=45)

Description:
1. B (Dry):
- BWh = Hot desert climate
- BWk = Cold desert climate
- BSh = Hot semi-arid climate
- BSk = Cold semi-arid climate
2. C (Temperate Climates):
- Cfa = Humid subtropical climate
- Cfb = Temperate oceanic climate or subtropical highland climate
- Csa = Hot-summer Mediterranean climate
- Csb = Warm-summer Mediterranean climate
3. D (Continental climates):
- Dfa = Hot-summer humid continental climate
- Dfb = Warm-summer humid continental climate
- Dwa = Monsoon-influenced hot-summer humid continental climate
- Dwb = Monsoon-influenced warm-summer humid continental climate
- Dsb = Mediterranean-influenced warm-summer humid continental climate
- Dsc = Mediterranean-influenced subarctic climate

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=data, x='lon', y='lat', hue = 'climateregions__climateregion', palette = 'Spectral')
plt.legend(loc='best')
plt.xlabel(xlabel='latitude')
plt.ylabel(ylabel='longitude')
plt.title(label='Locations of the climatic regions')

In [ ]:
# How many locations in each climatic region?

data['locations'] = data['lat'].astype(str) + '; ' + data['lon'].astype(str)
locations = data[['lat', 'lon', 'climateregions__climateregion', 'locations']].drop_duplicates()
locations.groupby('climateregions__climateregion', as_index=False, sort=False).aggregate({'locations' : 'count'})

In [ ]:
# transform climatic regions from categorical to numeric format to work further

labelencoder = LabelEncoder()
data['climateregions__climateregion'] = labelencoder.fit_transform(data['climateregions__climateregion'].values)

# We can visualize numeric climatic regions to be sure that we have got all 14 regions

data['climateregions__climateregion'].value_counts().plot(kind='bar', rot=45)

In [ ]:
# we can also transform locations from (lat; lon) to just numeric format like the climatic regions to simplify the work

data['locations'] = labelencoder.fit_transform(data['locations'].values)
data['locations']

### Features investigation and selection

For the modeling step, the environmental features were selected by the team through the provided EDA. That is why only these features takes into consideration.

In [ ]:
environmental_features = data[['startdate', 'lat', 'lon', 'elevation__elevation', 'climateregions__climateregion', 'locations',\
                           'contest-pevpr-sfc-gauss-14d__pevpr', 'contest-rhum-sig995-14d__rhum', \
                           'contest-slp-14d__slp', 'contest-pres-sfc-gauss-14d__pres', 'contest-prwtr-eatm-14d__prwtr',\
                           'contest-precip-14d__precip', 'contest-wind-h10-14d__wind-hgt-10',\
                     'contest-wind-h100-14d__wind-hgt-100', 'contest-wind-h500-14d__wind-hgt-500', \
                     'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-vwnd-250-14d__wind-vwnd-250',\
                     'contest-wind-vwnd-925-14d__wind-vwnd-925','contest-wind-uwnd-250-14d__wind-uwnd-250',\
                     'contest-wind-uwnd-925-14d__wind-uwnd-925', 'contest-tmp2m-14d__tmp2m']]

# rename some features for the clear description

environmental_features = environmental_features.rename(columns={'elevation__elevation': 'elevation', \
                                                                'climateregions__climateregion': 'climateregion',\
                                                               'contest-pevpr-sfc-gauss-14d__pevpr': 'evaporation',\
                                                               'contest-rhum-sig995-14d__rhum': 'humidity',\
                                                               'contest-slp-14d__slp': 'sea_level_pressure',\
                                                               'contest-pres-sfc-gauss-14d__pres': 'pressure',\
                                                               'contest-prwtr-eatm-14d__prwtr': 'precipitable_water',
                                                               'contest-precip-14d__precip': 'precipitation',\
                                                               'contest-wind-h10-14d__wind-hgt-10': 'wind_height_10',\
                                                               'contest-wind-h100-14d__wind-hgt-100': 'wind_height_100',\
                                                               'contest-wind-h500-14d__wind-hgt-500': 'wind_height_500',\
                                                               'contest-wind-h850-14d__wind-hgt-850': 'wind_height_850',\
                                                               'contest-wind-vwnd-250-14d__wind-vwnd-250': \
                                                                'long_wind_250', 'contest-wind-vwnd-925-14d__wind-vwnd-925':\
                                                               'long_wind_925', 'contest-wind-uwnd-250-14d__wind-uwnd-250':\
                                                               'zonal_wind_250', 'contest-wind-uwnd-925-14d__wind-uwnd-925':\
                                                               'zonal_wind_925', 'contest-tmp2m-14d__tmp2m': 'temperature'})

# five point summary

environmental_features.drop(columns={'startdate', 'lat', 'lon', 'climateregion', 'locations'}).describe().T

In [ ]:
plt.figure(figsize=(15,15))
corr_matrix = environmental_features.drop(columns={'startdate', 'lat', 'lon', 'climateregion', 'locations'}).corr()
sns.heatmap(corr_matrix, annot=True)

The heatmap shows that
1. we have high correlation between some features (|correlation|> 0.75):
- elevation and pressure = -0.92
- evaporation and humidity = -0.76
- evaporation and wind height 10 = 0.75
- wind height 10 and wind height 100 = 0.83
- evaporation and wind height 100 = 0.82
- evaporation and wind height 500 = 0.79
- wind height 100 and wind height 500 = 0.96
2. we have high correlation between temperature and 
- evaporation = 0.81
- precipitable water = 0.77
- wind height 10 = 0.76
- wind height 100 = 0.9
- wind height 500 = 0.88
3. we have low correlation (|correlation|< 0.5) between temperature and
- precipitation = 0.079
- elevation = -0.21
- pressure = 0.24
- longitudinal wind 250 = 0.43
- longitudinal wind 925 = 0.27
- zonal wind 250 = -0.33
- zonal wind 925 = -0.37

The variables must be independent between each other. Therefore the high correlated features should be excluded at the modeling stage. The low correlated features should be also excluded as the less informative.
We can remove wind height 10 and wind height 500. Although evaporation and temperature are high correlated we can also try to remove evaporation because of its high correlation with humidity and wind height 100, and observe what we will get.

In [ ]:
environmental_features_selected = environmental_features.drop(columns={'evaporation', 'wind_height_10', 'wind_height_500',\
                                                                       'elevation', 'precipitation', 'pressure',\
                                                                       'long_wind_250', 'long_wind_925', 'zonal_wind_250',
                                                                       'zonal_wind_925'})

In [ ]:
corr_matrix_selected = environmental_features_selected.drop(columns={'lat', 'lon', 'startdate', 'climateregion', 'locations'})\
.corr()
sns.heatmap(corr_matrix_selected, annot=True)

We have got 5 features.

In [ ]:
# features simple visualization

environmental_features_selected.drop(columns={'lat', 'lon', 'startdate', 'climateregion', 'locations'})\
.plot(figsize = (22, 16), subplots=True, layout=(len(environmental_features_selected.columns),1)) 

We have 514 locations and 14 climatic regions. The measurements are spread on the graphs through the whole time period.

To try modeling step and interpret the results correctly, we can start simply from one location. Location # 1 for instance.

### Single location

In [ ]:
location_number = 10 # choose the number of location
single_location = environmental_features_selected[environmental_features_selected['locations'] == location_number]
single_location.head()

In [ ]:
# features simple visualization

single_location.drop(columns={'lat', 'lon', 'startdate', 'climateregion', 'locations'})\
.plot(figsize = (22, 16), subplots=True, layout=(len(single_location.columns),1)) 

Now we can see variables dynamic for location # 1

and their distributions:

In [ ]:
single_location.drop(columns={'lat', 'lon', 'startdate', 'climateregion', 'locations'})\
.hist(figsize = (8, 30), layout=(len(single_location.columns),1)) 

### Features preparations to the modeling step

In [ ]:
# we take only date and necessary features for the prediction

single_location = single_location.drop(columns={'lat', 'lon', 'climateregion', 'locations'}).set_index('startdate')
single_location.head()

In [ ]:
single_location.shape

In [ ]:
# normalizing the features

scaler = MinMaxScaler()
dataset = scaler.fit_transform(single_location)

This tutorial was used for the futher work:
https://www.kaggle.com/code/mineshjethva/time-series-forecasting-with-lstm-for-uni-multivar/notebook

We are going to use a multivariate approach since we have 5 features and the temperature itself.

To lean by steps, firstly we will try to predict the values of the temperature for the next 14 days for one location by building LSTM model.

We have 731 observations and one measurement per day. 
In order to make this prediction, we choose to use 7 days of observations (is chosen via experiments for the model as an optimal one). 
Thus, we would create a window containing the last 7 observations to train the model.

In [ ]:
# The function below returns the above described windows of time for the model to train on

# history_size is the size of the past window of information
# target_size is how far in the future does the model need to learn to predict, is the label that needs to be predicted

def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
# "multivariate" means that we use several features (not one)
    data = [] # for x_train/validation
    labels = [] # for y_train/validation
    start_index = start_index + history_size
    
    if end_index is None: # for the validation part
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index): # sequences creation 
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        labels.append(target[i:i+target_size])
        
    return np.array(data), np.array(labels)

We do not use our test dataset and split the training dataset into training and validation parts to adjust the model.

In [ ]:
# setting the inputs for the function

training_size = int(0.8*dataset.shape[0]) # define the size of training part
start_index = 0
end_index = training_size
history_size = 7 # is the size of the past window of information
target_size = 14 # days, is how far in the future does the model need to learn to predict, 
#is the label that needs to be predicted
step = 1 # we have one measurement per day that is one day

In [ ]:
# splitting the dataset into training and validation parts and transforming it with the described window

x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, -1], start_index, end_index, history_size, target_size,\
                                                 step)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, -1], start_index, None, history_size, target_size, step)

Now we are ready to build a model.

### Building LSTM model

In [ ]:
# functions for plotting the results

# creating time steps for plotting
def create_time_steps(length):
    return list(range(-length, 0))

# plotting predictions
def multi_step_plot(history, true_future, prediction, title):
    plt.figure(figsize=(10, 5))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, -1]), label='History')
    plt.plot(np.arange(num_out)/step, np.array(true_future), 'bo', label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/step, np.array(prediction), 'ro', label='Predicted Future')
    plt.ylim(top=1, bottom=0)
    plt.legend(loc='lower left')
    plt.title(title)
    plt.savefig('prediction_plot.png', facecolor='w', edgecolor='w')
    
# plotting loss results    
def plot_train_history(history, title):
    # the loss is mse in the model configuration, and we calculate rmse by tf.sqrt for the task goal
    loss = tf.sqrt(history.history['loss']) 
    val_loss = tf.sqrt(history.history['val_loss'])

    epochs = range(len(loss))

    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.legend()
    plt.ylim(top=0.2)
    plt.title(title)
    plt.savefig('loss_plot.png', facecolor='w', edgecolor='w')

In [ ]:
# We use tensorflow and keras to work with LSTM

# perform shuffling, batching, and caching the dataset

BATCH_SIZE = 16 # is chosen via experiments for the model as an optimal one
# about batch size https://ai.stackexchange.com/questions/8560/how-do-i-choose-the-optimal-batch-size
BUFFER_SIZE = 10000 # 10000 in tutorial
# about buffer size 
#https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

In [ ]:
# constructing the model

multi_step_model = tf.keras.models.Sequential() # layers creating 
# the first layer
# activation functions are 'tanh' and recurrent_activation='sigmoid'
multi_step_model.add(tf.keras.layers.LSTM(64, # 32 neurons in tutorial
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:])) 
# the second layer
multi_step_model.add(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)) # 32 neurons, activation 'relu' in tutorial
multi_step_model.add(tf.keras.layers.LSTM(16, activation='softmax')) # 32 neurons, activation 'relu' in tutorial
# the output layer
multi_step_model.add(tf.keras.layers.Dense(14)) # since 14 predictions are made 
# the dense layer outputs 14 predictions

# configures the model for training
multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mse', metrics=['accuracy']) # learning_rate=0.001 as default
# clipvalue: If set, the gradient of each weight is clipped to be no higher than this value

In [ ]:
multi_step_model.summary()

In [ ]:
# how the model predicts before it trains

for x, y in val_data_multi.take(1): # .take () creates a dataset with at most count elements from this dataset
    # representing the number of elements of this dataset that should be taken to form the new dataset
    print (multi_step_model.predict(x).shape)

In [ ]:
multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0], 'How the model predicts before it trains')

In [ ]:
# training the model

EVALUATION_INTERVAL = training_size 
# steps per epoch is calculated as train_length // batch_size, 200 in tutorial (is said that complete training data is normal)
EPOCHS = 20 # 20 in tutorial

multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50) 
# validation_steps = total_validation_samples // validation_batch_size
# validation_batch_size If unspecified, will default to batch_size

In [ ]:
# the history and the future data are sampled every day
for x, y in train_data_multi.take(1):
    multi_step_plot(x[0], y[0], np.array([0]), 'True future values')

In [ ]:
# loss function results
plot_train_history(multi_step_history, 'Multi-Step training and validation loss, RMSE')

In [ ]:
# the history, the true and predicted future data are sampled every day
for x, y in val_data_multi.take(1): 
    multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0], 'How the model predicts after training')

In [ ]:
# plotting error distribution for 14 predictions
error = pd.DataFrame({'error':y[0] - multi_step_model.predict(x)[0]})
error = error.reset_index()
error = error.drop(columns=['index'])
error.plot.hist()
plt.ylim(top=7)
plt.title('Error distribution')
plt.savefig('error_distribution.png', facecolor='w', edgecolor='w')